In [5]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
import folium
import geopandas as gpd
from folium.plugins import HeatMap

data=pd.read_csv(r"/home/diego/Datasets/crimen_nac.csv", encoding="latin1")
map_mec=gpd.read_file(r"/home/diego/Datasets/mexicoHigh.json")
geojson_map_mec = folium.GeoJson(map_mec)
mapa_mexico = folium.Map(location=[23.6345, -102.5528], zoom_start=5)



In [6]:


data["Entidad"]=data["Entidad"].str.replace("Coahuila de Zaragoza","Coahuila")
data["Entidad"]=data["Entidad"].str.replace("Michoacán de Ocampo","Michoacán")
data["Entidad"]=data["Entidad"].str.replace("Veracruz de Ignacio de la Llave","Veracruz")

data_com=pd.merge(data, map_mec, left_on="Entidad", right_on="name", how="inner")
data_com["Total"]=data_com[data_com.columns.to_list()[7:19]].sum(axis=1)


In [7]:

ano=2019
delito="Homicidio"
sub_delito="Homicidio culposo"
Modalidad ="Con arma de fuego"

sub=data_com[(data_com["Año"]==ano) & (data_com["Tipo de delito"]==delito) & (data_com["Tipo de delito"]==delito) & (data_com["Subtipo de delito"]==sub_delito) & (data_com["Modalidad"]==Modalidad)]

In [10]:

heat_data=sub.groupby(["Entidad" ,"Tipo de delito", "Subtipo de delito", "Modalidad", "geometry"]).agg({"Total":"sum"}).reset_index()

eat_data = heat_data.apply(lambda x: [x.geometry.centroid.y, x.geometry.centroid.x, x['Total']], axis=1).tolist()
HeatMap(eat_data).add_to(mapa_mexico)
mapa_mexico